In [90]:
import pandas as pd
import pandas as pd
import numpy as np
from keras.utils import np_utils
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.preprocessing import MinMaxScaler
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV
import matplotlib.pyplot as plt
import itertools
from sklearn.metrics import classification_report, plot_confusion_matrix,fbeta_score, make_scorer
import seaborn as sns
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold
from sklearn.tree import DecisionTreeClassifier

import time

In [91]:
def print_distribution(arr, print_nl=True):
    arr = np.unique(arr, return_counts=True)[1] / len(arr)
    for i in range(arr.shape[0]):
        print("Classe %d: %.2f%%" %(i, arr[i]*100))

    if print_nl:
        print("\n")

In [99]:
def taste(quality):
  if quality <= 5:
    return 1
  elif quality > 5 and quality <= 7:
    return 2
  else:
    return 3

In [100]:
time_begin = time.time()
data = pd.read_csv("wine_quality.csv")
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6497 entries, 0 to 6496
Data columns (total 12 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   fixed.acidity         6497 non-null   float64
 1   volatile.acidity      6497 non-null   float64
 2   citric.acid           6497 non-null   float64
 3   residual.sugar        6497 non-null   float64
 4   chlorides             6497 non-null   float64
 5   free.sulfur.dioxide   6497 non-null   float64
 6   total.sulfur.dioxide  6497 non-null   float64
 7   density               6497 non-null   float64
 8   pH                    6497 non-null   float64
 9   sulphates             6497 non-null   float64
 10  alcohol               6497 non-null   float64
 11  quality               6497 non-null   int64  
dtypes: float64(11), int64(1)
memory usage: 609.2 KB


In [101]:
new_data = data
new_data['taste'] = data['quality'].apply(taste)

In [106]:
new_data

,fixed.acidity,volatile.acidity,citric.acid,residual.sugar,chlorides,free.sulfur.dioxide,total.sulfur.dioxide,density,pH,sulphates,alcohol,quality,taste
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.99780,3.51,0.56,9.4,5,1
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.99680,3.20,0.68,9.8,5,1
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.99700,3.26,0.65,9.8,5,1
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.99800,3.16,0.58,9.8,6,2
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.99780,3.51,0.56,9.4,5,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
6492,6.2,0.21,0.29,1.6,0.039,24.0,92.0,0.99114,3.27,0.50,11.2,6,2
6493,6.6,0.32,0.36,8.0,0.047,57.0,168.0,0.99490,3.15,0.46,9.6,5,1
6494,6.5,0.24,0.19,1.2,0.041,30.0,111.0,0.99254,2.99,0.46,9.4,6,2
6495,5.5,0.29,0.30,1.1,0.022,20.0,110.0,0.98869,3.34,0.38,12.8,7,2


## Divisão de base de dados e separação de base

In [103]:
#X = data[["fixed.acidity","volatile.acidity","citric.acid","residual.sugar","chlorides","free.sulfur.dioxide","total.sulfur.dioxide","density","pH","sulphates","alcohol"]]
#y = data[["quality"]]
X = new_data.iloc[:, 0:11].values
y = new_data.iloc[:, 12].values

In [104]:
print("Proporções por classe no dataset em geral:")
print_distribution(y)

kf = StratifiedKFold(n_splits=10, shuffle=True, random_state=199)
fold = 0
for train, test in kf.split(X, y):
    print("Fold %d" %(fold))
    print_distribution(y[train], print_nl=(fold != 2))
    fold += 1

Proporções por classe no dataset em geral:
Classe 0: 36.69%
Classe 1: 60.26%
Classe 2: 3.05%


Fold 0
Classe 0: 36.69%
Classe 1: 60.27%
Classe 2: 3.04%


Fold 1
Classe 0: 36.69%
Classe 1: 60.27%
Classe 2: 3.04%


Fold 2
Classe 0: 36.69%
Classe 1: 60.27%
Classe 2: 3.04%
Fold 3
Classe 0: 36.69%
Classe 1: 60.27%
Classe 2: 3.04%


Fold 4
Classe 0: 36.70%
Classe 1: 60.25%
Classe 2: 3.04%


Fold 5
Classe 0: 36.70%
Classe 1: 60.25%
Classe 2: 3.04%


Fold 6
Classe 0: 36.70%
Classe 1: 60.25%
Classe 2: 3.04%


Fold 7
Classe 0: 36.70%
Classe 1: 60.24%
Classe 2: 3.06%


Fold 8
Classe 0: 36.70%
Classe 1: 60.24%
Classe 2: 3.06%


Fold 9
Classe 0: 36.70%
Classe 1: 60.26%
Classe 2: 3.04%




In [107]:
kf_results = list()
kf_conf_mat = list()
kf_prec = list()
kf_f1 = list()
kf_rec = list()

from sklearn.metrics import make_scorer, f1_score,precision_score,recall_score
scores = {'accuracy': make_scorer(accuracy_score),
           'precision': make_scorer(precision_score, average = 'macro'),
           'recall': make_scorer(recall_score, average = 'macro'),
           'f1_macro': make_scorer(f1_score, average = 'macro'),
           'f1_weighted': make_scorer(f1_score, average = 'weighted')}

# Árvore de Decisão

In [ ]:
for train_ix, test_ix in kf.split(X,y):
    X_train, X_test = X[train_ix, :], X[test_ix, :]
    y_train, y_test = y[train_ix], y[test_ix]
    #cv_inner = StratifiedKFold(n_splits=5, shuffle=True, random_state=1)
    model = DecisionTreeClassifier(random_state=1)
    # define search space
    parameters = dict()
    parameters["max_leaf_nodes"] = list(range(2, 100))
    parameters["min_samples_split"] = [2, 4, 6]
    search = GridSearchCV(model, parameters, scoring="accuracy", cv=10, refit=True)
    # execute search
    result = search.fit(X_train, y_train)
    # get the best performing model fit on the whole training set
    best_model = result.best_estimator_
    # evaluate model on the hold out dataset
    yhat = best_model.predict(X_test)
    # evaluate the model
    acc = accuracy_score(y_test, yhat)
    conf_mat = confusion_matrix(y_test, yhat)
    precision = precision_score(y_test, yhat, average = 'macro')
    f1 = f1_score(y_test, yhat, average = 'weighted')
    recall = recall_score(y_test, yhat, average = 'macro')
    # store the result
#     kf_class_rep.append(class_rep)
    kf_conf_mat.append(conf_mat)
    kf_results.append(acc)
    kf_prec.append(precision)
    kf_f1.append(f1)
    kf_rec.append(recall)
    # report progress
    print('>acc=%.3f,>prec=%.3f,>f1=%.3f,>rec=%.3f est=%.3f, cfg=%s' % (acc,precision,f1,recall, result.best_score_, result.best_params_))
    print(conf_mat)
#     print(class_rep)

C:\Users\cfpc2\anaconda3\envs\rv\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


>acc=0.715,>prec=0.469,>f1=0.702,>rec=0.476 est=0.687, cfg={'max_leaf_nodes': 6, 'min_samples_split': 2}
[[147  92   0]
 [ 73 318   0]
 [  0  20   0]]


C:\Users\cfpc2\anaconda3\envs\rv\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


>acc=0.685,>prec=0.448,>f1=0.667,>rec=0.447 est=0.694, cfg={'max_leaf_nodes': 6, 'min_samples_split': 2}
[[124 115   0]
 [ 70 321   0]
 [  0  20   0]]


C:\Users\cfpc2\anaconda3\envs\rv\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


...

>acc=0.723,>prec=0.474,>f1=0.710,>rec=0.484 est=0.692, cfg={'max_leaf_nodes': 6, 'min_samples_split': 2}
[[153  85   0]
 [ 75 317   0]
 [  1  19   0]]


C:\Users\cfpc2\anaconda3\envs\rv\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


>acc=0.675,>prec=0.441,>f1=0.651,>rec=0.431 est=0.689, cfg={'max_leaf_nodes': 6, 'min_samples_split': 2}
[[106 132   0]
 [ 60 332   0]
 [  1  18   0]]


C:\Users\cfpc2\anaconda3\envs\rv\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


>acc=0.681,>prec=0.443,>f1=0.669,>rec=0.452 est=0.693, cfg={'max_leaf_nodes': 6, 'min_samples_split': 2}
[[139  99   0]
 [ 89 303   0]
 [  0  19   0]]


In [74]:
print('Accuracy: %.3f (%.3f)' % (np.mean(kf_results), np.std(kf_results)))
print('Precision: %.3f (%.3f)' % (np.mean(kf_prec), np.std(kf_prec)))
print('F1-score: %.3f (%.3f)' % (np.mean(kf_f1), np.std(kf_f1)))
print('Recall: %.3f (%.3f)' % (np.mean(kf_rec), np.std(kf_rec)))
#print('Confusion Matrix: %.3f (%.3f)' % (np.mean(kf_conf_mat)))

Accuracy: 0.522 (0.013)
Precision: 0.167 (0.015)
F1-score: 0.451 (0.012)
Recall: 0.208 (0.016)


In [54]:
kf_conf_mat

[array([[  0,   0,   1,   2,   0,   0],
        [  0,   0,  10,  11,   0,   0],
        [  0,   0, 139,  75,   0,   0],
        [  0,   0,  76, 208,   0,   0],
        [  0,   0,   3, 105,   0,   0],
        [  0,   0,   1,  19,   0,   0]], dtype=int64),
 array([[  0,   0,   1,   2,   0,   0],
        [  0,   0,   6,  15,   0,   0],
        [  0,   0, 117,  97,   0,   0],
        [  0,   0,  51, 233,   0,   0],
        [  0,   0,   5, 103,   0,   0],
        [  0,   0,   0,  20,   0,   0]], dtype=int64),
 array([[  0,   0,   0,   3,   0,   0,   0],
        [  0,   0,   9,  12,   0,   0,   0],
        [  0,   0, 115,  99,   0,   0,   0],
        [  0,   0,  68, 216,   0,   0,   0],
        [  0,   0,   0, 108,   0,   0,   0],
        [  0,   0,   0,  19,   0,   0,   0],
        [  0,   0,   0,   1,   0,   0,   0]], dtype=int64),
 array([[  0,   0,   2,   1,   0,   0,   0],
        [  0,   0,  11,  11,   0,   0,   0],
        [  0,   0, 149,  65,   0,   0,   0],
        [  0,   0,  96, 1

In [55]:
teste = kf_conf_mat[0:2]
teste.append(kf_conf_mat[7:10])
np.mean(teste)
# teste

array([[[  0,   0,   1,   2,   0,   0],
        [  0,   0,   8,  12,   0,   0],
        [  0,   0, 128,  85,   0,   0],
        [  0,   0,  70, 213,   0,   0],
        [  0,   0,   6, 102,   0,   0],
        [  0,   0,   0,  19,   0,   0]],

       [[  0,   0,   1,   2,   0,   0],
        [  0,   0,   8,  13,   0,   0],
        [  0,   0, 127,  86,   0,   0],
        [  0,   0,  68, 216,   0,   0],
        [  0,   0,   4, 103,   0,   0],
        [  0,   0,   0,  19,   0,   0]],

       [[  0,   0,   1,   2,   0,   0],
        [  0,   0,   9,  11,   0,   0],
        [  0,   0, 117,  96,   0,   0],
        [  0,   0,  59, 224,   0,   0],
        [  0,   0,   3, 104,   0,   0],
        [  0,   0,   0,  19,   0,   0]]], dtype=int64)

In [22]:
time_end = time.time()
print((time_end - time_begin)/60)

24.052873226006827
